# Section 1
You would have received a Access_data.zip file with 22 datasets in .csv format and 1 data dictionary in .docx format. The datasets are sample of the mock-data on individuals accessing two office sites of Company ABC, consisting of:

1. When: Time of entry by the individual

2. Profile: Type of access card
    -	0 - Staff Pass
    -	1 - Temp Pass
    -	2 - Visitor Pass

3. Dept: Department of the individual

4. CardNum: Card unique identifier. The length of the card number cannot be less than 8 characters. Currently, if CardNum starts with a/multiple ‘0’, the data captured in system will exclude/remove the “0”.

You can assume that the **total number of staff in the company is 2000 and the data is extracted from the company’s building access system.** An individual can tap in and out several times within the same day. When the individual first clock in, that would be the earliest time slot and the only record you will base off the analysis. (You can also state your other assumptions if need be.)


In [1]:
## Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels as sm
import os

## Question 1.1
Write code preferably in R or Python to process and **organise the raw data (Access_data.zip) to make it suitable for analysis**. Identify and resolve the data quality issues in the raw data, if any. (The created code should allow user to efficiently and easily run it to ingest additional datasets of different period, beyond the given sample.)

We assume the files are represented in the format of 'SiteAYYYYMMDD-YYYYMMDDa.csv' or 'SiteBYYYYMMDD-YYYYMMDDb.csv' for both sites respectively.

In [2]:
# Utility function for listing particular required site files in a specified data directory path
def list_files_for_a_site(site_name):
    data_dir_path=os.path.join(os.getcwd(), "data", "Access_Data")
    return [os.path.join(data_dir_path, file) for file in os.listdir(data_dir_path) if file.startswith(site_name)]

In [3]:
# Construct list of site A and site B files.
site_A_file_list =  list_files_for_a_site(site_name="SiteA")
site_B_file_list =  list_files_for_a_site(site_name="SiteB")

print(site_A_file_list)
print(site_B_file_list)

['c:\\Users\\Admin\\Desktop\\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\\data\\Access_Data\\SiteA20200420-20200426a.csv', 'c:\\Users\\Admin\\Desktop\\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\\data\\Access_Data\\SiteA20200427-20200503a.csv', 'c:\\Users\\Admin\\Desktop\\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\\data\\Access_Data\\SiteA20200504-20200510a.csv', 'c:\\Users\\Admin\\Desktop\\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\\data\\Access_Data\\SiteA20200511-20200517a.csv', 'c:\\Users\\Admin\\Desktop\\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\\data\\Access_Data\\SiteA20200518-20200524a.csv', 'c:\\Users\\Admin\\Desktop\\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\\data\\Access_Data\\SiteA20200525-20200531a.csv', 'c:\\Users\\Admin\\Desktop\\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\\data\\Access_Data\\SiteA20200601a.csv', 'c:\\Users\\Admin\\Desktop\\MOM_Senior-Analyst-Analyst_SensingAnalytics_Asses

In [4]:
def column_checker(df, filename):
    expected_col = set(["When", "Profile", "Dept", "CardNum"])
    symmetric_diff_set = set(df.columns).symmetric_difference(expected_col)
    if symmetric_diff_set :
        print(f"Identified a non-expected column for {filename}")
        print(f"Symmetric difference: {symmetric_diff_set}")
    return None

### Process site A and site B

Do a quick check on column name and found that one of site A data file has column named "Depts" instead of "Dept" while for site B, there is a column named "CardNum " instead of "CardNum" for same period of 20200622-20200628. To resolve this issue, we will strip all leading/trailing space and extract the first 4 alphanumeric representation for convenience

In [5]:
for file in site_A_file_list:
    temp_df = pd.read_csv(file, sep=",")
    column_checker(temp_df, file)

print()
for file in site_B_file_list:
    temp_df = pd.read_csv(file, sep=",")
    column_checker(temp_df, file)

Identified a non-expected column for c:\Users\Admin\Desktop\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\data\Access_Data\SiteA20200622-20200628a.csv
Symmetric difference: {'Depts', 'Dept'}

Identified a non-expected column for c:\Users\Admin\Desktop\MOM_Senior-Analyst-Analyst_SensingAnalytics_Assessment\data\Access_Data\SiteB20200622-20200628b.csv
Symmetric difference: {'CardNum ', 'CardNum'}


## Data loading
Load the each site's access data files as a dataframe into a list for vertical stacking. We assume the time period for both sites will be the period between Apr 20, 2020 to Jun 28, 2020 as per the on file name date representation.

Notice that there are quite a significant number of nulls for Department feature in Site A (9266) and Site B (10100); and 5 null card information for site B.

In [6]:
site_A_df_list = []
for file in site_A_file_list:
    temp_df = pd.read_csv(file, sep=",")
    temp_df.columns = [col.strip()[:4] for col in temp_df.columns]
    site_A_df_list.append(temp_df)

site_B_df_list = []
for file in site_B_file_list:
    temp_df = pd.read_csv(file, sep=",")
    temp_df.columns = [col.strip()[:4] for col in temp_df.columns]
    site_B_df_list.append(temp_df)


site_A_df = pd.concat(site_A_df_list, ignore_index=True)
site_B_df = pd.concat(site_B_df_list, ignore_index=True)
print(site_A_df.info())
print(site_B_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12192 entries, 0 to 12191
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   When    12192 non-null  object
 1   Prof    12192 non-null  object
 2   Dept    2925 non-null   object
 3   Card    12192 non-null  object
dtypes: object(4)
memory usage: 381.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24499 entries, 0 to 24498
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   When    24499 non-null  object
 1   Prof    24499 non-null  object
 2   Dept    14399 non-null  object
 3   Card    24494 non-null  object
dtypes: object(4)
memory usage: 765.7+ KB
None


## Managing Department and Profile features - Site A
Check Department and Profile uniqueness for both sides as they are categorical. Notice that for Site A, we see that there is some form of discrepancy for Dept 1, as well as the Profile value represented in numeric or string format.

To simplify department representation, we will remove all spaces and concatenate the alphanumeric representation, while for profile representation, we will do a string cast and standardise to meaningful name representation

In [7]:
# Convert card type to string in case of int and string mix representation
site_A_df["Card"] = site_A_df["Card"].astype(str)
print(site_A_df["Dept"].unique())
print(site_A_df["Prof"].unique())

[nan 'Dept 5' 'Dept 11' 'Dept 18' 'Dept 4' 'Dept 9' 'Dept 15' 'Dept 14'
 'Dept 2' 'Dept 8' 'Dept 12' 'Dept  1' 'Dept 1' 'Dept 19' 'Dept 7'
 'Dept 17' 'Dept 10' 'Dept 6' 'Dept 3' 'Dept 16' 'Dept 13']
[2 1 0 '1' '0' '2' 'Visitor Pass']


In [8]:
profile_pass_mapping = {
    "0": "Staff Pass",
    "1": "Temp Pass",
    "2": "Visitor Pass"
}

site_A_df["Dept"] = site_A_df["Dept"].fillna("unknown")
site_A_df["Dept"] = site_A_df["Dept"].map(lambda x: x.replace(" ","") if x else x)

site_A_df["Prof"] = site_A_df["Prof"].map(lambda x: str(x))
site_A_df["Prof"] = site_A_df["Prof"].map(lambda x: profile_pass_mapping[x] if x in profile_pass_mapping else x)

In [9]:
print(site_A_df["Dept"].unique())
print(site_A_df["Prof"].unique())

['unknown' 'Dept5' 'Dept11' 'Dept18' 'Dept4' 'Dept9' 'Dept15' 'Dept14'
 'Dept2' 'Dept8' 'Dept12' 'Dept1' 'Dept19' 'Dept7' 'Dept17' 'Dept10'
 'Dept6' 'Dept3' 'Dept16' 'Dept13']
['Visitor Pass' 'Temp Pass' 'Staff Pass']


## Null data handling: Site A
Handle missing data for Site A involving department feature. Identify the pass type (profile) which the department information is unknown. We need to check from the perspective of each pass type(profile) and see the applicability of department before deciding how to impute.

Checking from this 2 perspective, we conclude that the department information is not applicable for both temp/visitor pass. We can fill as not applicable representation.

In [10]:
# Identify the pass type (profile) which the department information is unknown, as we need to impute
site_A_df[site_A_df["Dept"]=="unknown"]["Prof"].unique()

array(['Visitor Pass', 'Temp Pass'], dtype=object)

In [11]:
# For each pass type (profile), see its applicability to department information
for profile in site_A_df["Prof"].unique():
    unique_dept = site_A_df[site_A_df["Prof"]==profile]["Dept"].unique()
    print(f"{profile}:{unique_dept}")

Visitor Pass:['unknown']
Temp Pass:['unknown']
Staff Pass:['Dept5' 'Dept11' 'Dept18' 'Dept4' 'Dept9' 'Dept15' 'Dept14' 'Dept2'
 'Dept8' 'Dept12' 'Dept1' 'Dept19' 'Dept7' 'Dept17' 'Dept10' 'Dept6'
 'Dept3' 'Dept16' 'Dept13']


## Managing Department and Profile features - Site B
Check Department and Profile uniqueness for both sides as they are categorical. Notice that for Site B, we see that the Profile value represented in numeric or string format.

To simplify department representation, we will use the same approach for this site remove all spaces and concatenate the alphanumeric representation, while for profile representation, we will do a string cast and standardise to meaningful name representation.

In [12]:
# Convert card type to string in case of int and string mix representation
site_B_df["Card"] = site_B_df["Card"].astype(str)
print(site_B_df["Dept"].unique())
print(site_B_df["Prof"].unique())

['Dept 4' nan 'Dept 2' 'Dept 16' 'Dept 10' 'Dept 5' 'Dept 14' 'Dept 3'
 'Dept 15' 'Dept 8' 'Dept 13' 'Dept 19' 'Dept 11' 'Dept 9' 'Dept 18'
 'Dept 7' 'Dept 17' 'Dept 1' 'Dept 12' 'Dept 6']
[0 1 2 '0' '1' '2' 'Temp Pass' 'Staff Pass']


Convert card value as string results in null value resolved.

In [27]:
site_B_df["Card"].notnull().all()

True

In [13]:
site_B_df["Dept"] = site_B_df["Dept"].fillna("unknown")
site_B_df["Dept"] = site_B_df["Dept"].map(lambda x: x.replace(" ","") if x else x)

site_B_df["Prof"] = site_B_df["Prof"].map(lambda x: str(x))
site_B_df["Prof"] = site_B_df["Prof"].map(lambda x: profile_pass_mapping[x] if x in profile_pass_mapping else x)

In [14]:
print(site_B_df["Dept"].unique())
print(site_B_df["Prof"].unique())

['Dept4' 'unknown' 'Dept2' 'Dept16' 'Dept10' 'Dept5' 'Dept14' 'Dept3'
 'Dept15' 'Dept8' 'Dept13' 'Dept19' 'Dept11' 'Dept9' 'Dept18' 'Dept7'
 'Dept17' 'Dept1' 'Dept12' 'Dept6']
['Staff Pass' 'Temp Pass' 'Visitor Pass']


Handle missing data for Site B involving Department feature.

For Department, identify the profile which department info is unknown. We also need to check from the perspective of each pass type(profile) and see the uniqueness of department to decide how to impute.

Checking from this 2 perspective, we conclude that the department information is not applicable for both temp/visitor pass. We can fill as not applicable representation.

In [15]:
site_B_df[site_B_df["Dept"]=="unknown"]["Prof"].unique()

array(['Temp Pass', 'Visitor Pass'], dtype=object)

In [16]:
for profile in site_B_df["Prof"].unique():
    unique_dept = site_B_df[site_B_df["Prof"]==profile]["Dept"].unique()
    print(f"{profile}:{unique_dept}")

Staff Pass:['Dept4' 'Dept2' 'Dept16' 'Dept10' 'Dept5' 'Dept14' 'Dept3' 'Dept15'
 'Dept8' 'Dept13' 'Dept19' 'Dept11' 'Dept9' 'Dept18' 'Dept7' 'Dept17'
 'Dept1' 'Dept12' 'Dept6']
Temp Pass:['unknown']
Visitor Pass:['unknown']


In both sites, we are sure that temp and visitor pass do not have corresponding department info. As such we will replace it with 'not_applicable' value instead of 'unknown' as a meaningful representation.

In [17]:
site_A_df["Dept"] = site_A_df["Dept"].map(lambda x:x.replace("unknown","not_applicable" if x=="unknown" else x))
site_B_df["Dept"] = site_B_df["Dept"].map(lambda x:x.replace("unknown","not_applicable" if x=="unknown" else x))

## Managing card feature 
We still have null cases where card information is unknown. A quick check shows the card affected are either temp pass/visitor pass. We may want to combine the dataframe from 2 sites to resolve the card information issue as no additional information are provided.

In [18]:
print(site_A_df.shape, site_B_df.shape)

(12192, 4) (24499, 4)


In [46]:
site_B_df["Site"] = "B"
site_A_df["Site"] = "A"

In [47]:
# Stack the dataframe vertically
both_sites_df = pd.concat([site_A_df, site_B_df], ignore_index=True)
both_sites_df.head()

,When,Prof,Dept,Card,Site
0,20/4/2020 7:17,Visitor Pass,not_applicable,1001,A
1,21/4/2020 7:10,Visitor Pass,not_applicable,1001,A
2,22/4/2020 7:09,Visitor Pass,not_applicable,1001,A
3,23/4/2020 7:16,Visitor Pass,not_applicable,1001,A
4,24/4/2020 7:25,Visitor Pass,not_applicable,1001,A


In [48]:
both_sites_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36691 entries, 0 to 36690
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   When    36691 non-null  object
 1   Prof    36691 non-null  object
 2   Dept    36691 non-null  object
 3   Card    36691 non-null  object
 4   Site    36691 non-null  object
dtypes: object(5)
memory usage: 1.4+ MB


In [49]:
# Check formatting of When feature
pd.to_datetime(both_sites_df["When"], format="%d/%m/%Y %H:%M", errors="coerce").notnull().all()

True

In [50]:
# Convert to datetime and extract date and time info separately
both_sites_df["When"] = pd.to_datetime(both_sites_df["When"], format="%d/%m/%Y %H:%M")
# Split When into Date and TIme
both_sites_df['Date'] = both_sites_df['When'].dt.date
both_sites_df['Time'] = both_sites_df['When'].dt.time

### Investigate card number in detail after typecasting into string as it may not be numeric.

Segregate data into numeric and non numeric dataframes based on card info.
A quick check shows 871 of such records.

In [51]:
both_sites_df["is_numeric_Card"] = both_sites_df["Card"].map(lambda x: x.isnumeric())

In [52]:
non_numeric_card_records_df = both_sites_df[both_sites_df["is_numeric_Card"]==False]
numeric_card_records_df = both_sites_df[both_sites_df["is_numeric_Card"]==True]
non_numeric_card_records_df

,When,Prof,Dept,Card,Site,Date,Time,is_numeric_Card
1036,2020-04-20 08:32:00,Staff Pass,Dept19,9 42530,A,2020-04-20,08:32:00,False
5853,2020-05-28 09:20:00,Staff Pass,Dept15,#VALUE!,A,2020-05-28,09:20:00,False
5854,2020-05-27 09:00:00,Staff Pass,Dept15,#VALUE!,A,2020-05-27,09:00:00,False
5855,2020-05-28 08:12:00,Temp Pass,not_applicable,#VALUE!,A,2020-05-28,08:12:00,False
6001,2020-06-01 08:14:00,Staff Pass,Dept2,#REF!,A,2020-06-01,08:14:00,False
...,...,...,...,...,...,...,...,...
25958,2020-05-26 08:31:00,Staff Pass,Dept16,#VALUE!,B,2020-05-26,08:31:00,False
28624,2020-06-03 09:34:00,Staff Pass,Dept8,#VALUE!,B,2020-06-03,09:34:00,False
28625,2020-06-07 09:17:00,Staff Pass,Dept8,#VALUE!,B,2020-06-07,09:17:00,False
28626,2020-06-04 09:10:00,Temp Pass,not_applicable,#VALUE!,B,2020-06-04,09:10:00,False


In [53]:
# Check dates which such records are affected
non_numeric_card_records_df["Date"].unique()

array([datetime.date(2020, 4, 20), datetime.date(2020, 5, 28),
       datetime.date(2020, 5, 27), datetime.date(2020, 6, 1),
       datetime.date(2020, 4, 21), datetime.date(2020, 4, 25),
       datetime.date(2020, 4, 23), datetime.date(2020, 4, 22),
       datetime.date(2020, 4, 24), datetime.date(2020, 4, 26),
       datetime.date(2020, 4, 30), datetime.date(2020, 4, 28),
       datetime.date(2020, 5, 3), datetime.date(2020, 5, 4),
       datetime.date(2020, 5, 7), datetime.date(2020, 5, 6),
       datetime.date(2020, 5, 8), datetime.date(2020, 5, 29),
       datetime.date(2020, 5, 26), datetime.date(2020, 6, 3),
       datetime.date(2020, 6, 7), datetime.date(2020, 6, 4),
       datetime.date(2020, 6, 23)], dtype=object)

In [36]:
# Check card numbers affected
non_numeric_card_records_df["Card"].value_counts()

Card
#REF!      846
#VALUE!     18
nan          5
9 42530      1
40202!       1
Name: count, dtype: int64

In [54]:
# Get the breakdown of profile, department and site which card info is non numeric
non_numeric_card_records_df[["Prof", "Dept", "Site"]].value_counts()

Prof          Dept            Site
Temp Pass     not_applicable  B       745
                              A        48
Staff Pass    Dept18          A        13
              Dept15          A        10
              Dept5           A        10
              Dept8           B        10
              Dept14          A         5
              Dept2           A         5
              Dept11          A         5
              Dept8           A         4
              Dept16          B         4
              Dept17          A         3
              Dept3           A         2
Visitor Pass  not_applicable  B         2
Staff Pass    Dept7           A         2
              Dept4           A         1
              Dept19          A         1
              Dept10          A         1
Name: count, dtype: int64

In [ ]:
# Try identifying the card ID for each department basedon staff pass by using daily tap in records

for date in both_sites_df["Date"].unique():
    for dept in both_sites_df["Dept"].unique():
        both_sites_df[(both_sites_df["Dept"] == dept) & (both_sites_df["Date"]== "Date")]
        unique_staff_cards_list = both_sites_df[]
        print("")

In [38]:
numeric_card_records_df["Card"].nunique()

2443

Question 1.2
Using the prepared data from question 1.1, create dashboard(s) for end-users to self-serve to report and make decisions on the following issues on a regular basis.

a) The company’s target is to have more than 80% of staff working from home at any time.

b) For those staff who are not able to work from home, the company wants to know how many days per week are they coming in and to stagger their work hours and ensure an even proportion are coming in at the three different time slots. The proportion should roughly be 1/3 for each slot.

i. 0700 to 0829

ii. 0830 to 0929

iii. 0930 to 1030

c) For individuals issued with Temp or Visitor Pass, the company wants to know the number at each office site.